# Artificial Neural Network  

Table of Contents:

- [Modeling one neuron](#model)
  - [Biological motivation](#bio)
  - [Single neuron as a linear classifier](#classifier)
  - [Activation functions](#actfun)
- [Neural Network architectures](#nn)  
  - [Layer-wise organization](#layers)
  - [Example feed-forward computation](#feedforward)
  - [Representational power](#power)
  - [Setting number of layers and their sizes](#arch)
- [Summary](#summary)
- [Additional references](#add)

## Modeling One Neuron <a name='model'></a>
### Biological motivation <a name='bio'></a>
![neuro-model](./figures/ann/neuron-model.png)

The basic computational unit of the brain is a **neuron**.  
**Activation function**  $f$ represents the frequency of the spikes along the axon.  
In the computational model, each neuron performs a dot product with the input and its weights, adds the bias and applies the **non-linearity** (or activation function).

### Single neuron as a linear classifer <a name='classifier'></a>
With an appropriate loss function on the neuron's output, we can turn a single neuron into a linear classifier:
- **Binary Softmax classifier**. For example, **logistic regression**. The ouput of the neuron is the logarithm loss.  
- **Binary SVM classifier**. The output of the neuron is the hinge loss. 

**Regularization interpretation**. The regularization loss in both SVM and Softmax cases could in this biological view be interpreted as *gradual forgetting*, since it would have the effect of driving all synaptic weights \\(w\\) towards zero after every parameter update.

> A single neuron can be used to implement a binary classifier (e.g. binary Softmax or binary SVM classifiers)

In [1]:
class Neuron(object):
  # ... 
  def forward(self, inputs):
    """ assume inputs and weights are 1-D numpy arrays and bias is a number """
    cell_body_sum = np.sum(inputs * self.weights) + self.bias
    firing_rate = 1.0 / (1.0 + math.exp(-cell_body_sum)) # sigmoid activation function
    return firing_rate

### Activation functions <a name='actfun'></a>
Every activation function (or non-linearity) takes a single number and performs a certain fixed mathematical operation on it.
![sigma-tanh](./figures/ann/sigma-tanh.png)
- **Sigmoid.** The sigmoid non-linearity has the mathematical form 
$$
\sigma(x) = \frac{1} {1 + e^{-x}}.
$$
In practice, the sigmoid non-linearity is rarely ever used.
  - (-) Sigmoids saturate and kill gradients. When the neuron's activation saturates at either tail of 0 or 1, the gradient at these regions is almost zero.
  - (-) Sigmoid outputs are not zero-centered. If the data coming into a neuron is always positive (e.g. $x > 0$ elementwise in $f = w^Tx + b$), then the gradient on the weights $w$ will during backpropagation become either all be positive, or all negative introducing undesirable zig-zagging dynamics in the gradient updates for the weights. However, once these gradients are added up, the issue is mitigated.
  - (-) $\exp()$ computationaly expensive
- **Tanh.** The tanh non-linearity has the following mathematical form
$$
tanh(x) = 2\sigma(2x)-1.
$$
  - (+) Activations saturate
  - (-) Output is zero-centered.
  
  Therefore, in practice the *tanh non-linearity is always preferred to the sigmoid nonlinearity.*
- **ReLU.** The Rectified Linear Unit has become very popular in the last few years. It computes the function 
$$
f(x) = \max(0, x).
$$
In other words, the activation is simply thresholded at zero. There are several pros and cons to using the ReLUs: 
  - (+) Greatly accelerate (e.g. a factor of 6 in [Krizhevsky et al.](http://www.cs.toronto.edu/~fritz/absps/imagenet.pdf)) the convergence of stochastic gradient descent compared to the sigmoid/tanh functions. It is argued that this is due to its linear, non-saturating form.
  - (+) Compared to tanh/sigmoid neurons that involve expensive operations (exponentials, etc.), the ReLU can be implemented by simply thresholding a matrix of activations at zero.
  - (-) ReLU units can be fragile during training and can "die". The ReLU units can irreversibly die during training since they can get knocked off the data manifold. 
  In practice, people like to initialize ReLU neurons with slightly positive biases (e.g. 0.01)
- **Leaky ReLU.** Leaky ReLUs are one attempt to fix the "dying ReLU" problem. The function computes
$$
f(x) = \mathbb{1}(x < 0) (\alpha x) + \mathbb{1}(x>=0) (x)
$$
where $\alpha$ is a small constant. However, the consistency of the benefit across tasks is presently unclear.
- **Maxout.** Maxout neuron (introduced by [Goodfellow et al.](http://www-etud.iro.umontreal.ca/~goodfeli/maxout.html)) that generalizes the ReLU and its leaky version :
$$
\max(w_1^Tx+b_1, w_2^Tx + b_2).
$$
Notice that both ReLU and Leaky ReLU are a special case of this form (for example, for ReLU we have $w_1, b_1 = 0$). The Maxout neuron therefore enjoys all the benefits of a ReLU unit (linear regime of operation, no saturation) and does not have its drawbacks (dying ReLU). However, unlike the ReLU neurons it doubles the number of parameters for every single neuron, leading to a high total number of parameters. 

**Conclusion:**
- Use the ReLU non-linearity, be careful with learning rates and possibly monitor the fraction of "dead" units in a network. 
- If bother with "dead" units, give Leaky ReLU or Maxout a try. 
- Never use sigmoid. 
- Try tanh, but expect it to work worse than ReLU/Maxout.

## Neural network architectures <a name='nn'></a>
### Layer-wise organization <a name='layers'></a>
**Neural Networks** are modeled as collections of neurons that are connected in an acyclic graph. Cycles are not allowed since that would imply an infinite loop in the forward pass of a network. Neural Network models are often organized into distinct layers of neurons. In a **Fully-connected layer**, neurons between two adjacent layers are fully pairwise connected, but neurons within a single layer share no connections.
![neuro-network](./figures/ann/neural-network-architechtures.png)
**Naming conventions.** When we say N-layer neural network, we do not count the input layer. Therefore, a single-layer neural network describes a network with no hidden layers (input directly mapped to output).

**Output layer.** The output layer neurons most commonly do not have an activation function (or you can think of them as having a linear identity activation function). The last output layer is usually taken to represent the class scores (e.g. in classification), which are arbitrary real-valued numbers, or some kind of real-valued target (e.g. in regression). 

**Sizing neural networks**. The number of neurons, or more commonly the number of parameters are two metrics used to measure the size of neural networks.

- The network shown in the above picture right has 4 + 4 + 1 = 9 neurons, [3 x 4] + [4 x 4] + [4 x 1] = 12 + 16 + 4 = 32 weights and 4 + 4 + 1 = 9 biases, for a total of 41 learnable parameters.

Modern Convolutional Networks contain on orders of 100 million parameters and are usually made up of approximately 10-20 layers (hence **deep learning**).

### Example feed-forward computation <a name='feedforward'></a>
Repeated matrix multiplications interwoven with activation function. One of the primary reasons that Neural Networks are organized into layers is that this structure makes it very simple and efficient to evaluate Neural Networks using **matrix vector operations**.

In [15]:
# forward-pass of a 3-layer neural network:
import numpy as np

f = lambda x: 1.0/(1.0 + np.exp(-x)) # activation function (use sigmoid)
x = np.random.randn(3, 1) # random input vector of three numbers (3x1)
W1 = 0.01*np.random.randn(4,3)
W2 = 0.01*np.random.randn(4,4)
W3 = 0.01*np.random.randn(1,4)
b1 = b2 = np.zeros((4,1))
b3 = 0
h1 = f(np.dot(W1, x) + b1) # calculate first hidden layer activations (4x1)
h2 = f(np.dot(W2, h1) + b2) # calculate second hidden layer activations (4x1)
out = np.dot(W3, h2) + b3 # output neuron (1x1)
print(out)

[[-0.0063565]]


Notice that the variable `x` could hold an entire batch of training data (where each input example would be a column of `x`) and then all examples would be efficiently evaluated in parallel.
> The forward pass of a fully-connected layer corresponds to one matrix multiplication followed by a bias offset and an activation function.

### Representational power <a name='power'></a>
It turns out that Neural Networks with at least one hidden layer are **universal approximators**. In other words, the neural network can approximate any continuous function ([*Approximation by Superpositions of Sigmoidal Function*](http://www.dartmouth.edu/~gvc/Cybenko_MCSS.pdf), or this [intuitive explanation](http://neuralnetworksanddeeplearning.com/chap4.html)). 

The fact that deeper networks (with multiple hidden layers) can work better than a single-hidden-layer networks is an empirical observation, despite the fact that their representational power is equal.

In practice it is often the case that 3-layer neural networks will outperform 2-layer nets, but going even deeper (4,5,6-layer) rarely helps much more. As for Convolutional Networks, depth has been found to be an extremely important component for a good recognition system (e.g. on order of 10 learnable layers).

Further reading for these topics:
- [Deep Learning](http://www.deeplearningbook.org/) book, in particular [Chapter 6.4](http://www.deeplearningbook.org/contents/mlp.html).
- [Do Deep Nets Really Need to be Deep?](http://arxiv.org/abs/1312.6184)
- [FitNets: Hints for Thin Deep Nets](http://arxiv.org/abs/1412.6550)

### Setting number of layers and their sizes <a name='arch'></a>
Note that as we increase the size and number of layers in a Neural Network, the **capacity** of the network increases.
![layer-sizes](./figures/ann/layer_sizes.png)

Shown in the diagram above (<a href="http://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html">ConvNetsJS demo</a>), Neural Networks with more neurons can express more complicated functions. However, it is easier to **overfit** the training data. The model with 3 hidden neurons models the data as two blobs and interprets the few red points inside the green cluster as **outliers** (noise). In practice, this could lead to better **generalization** on the test set.

It seems that smaller neural networks can be preferred if the data is not complex enough to prevent overfitting. However, in practice, it is always better to use methods such as L2 regularization, dropout, input noise to control overfitting instead of the number of neurons. 

The subtle reason behind this is that smaller networks are harder to train with local methods such as Gradient Descent. Conversely, bigger neural networks contain significantly more local minima, but these minima turn out to be much better in terms of their actual loss ([The Loss Surfaces of Multilayer Networks](http://arxiv.org/abs/1412.0233)). In practice, a small network the final loss can display a good amount of variance.

To reiterate, the regularization strength is the preferred way to control the overfitting of a neural network.
![reg-strengths](./figures/ann/reg_strengths.png)
Shown in the diagram above (<a href="http://cs.stanford.edu/people/karpathy/convnetjs/demo/classify2d.html">ConvNetsJS demo</a>), each neural network above has 20 hidden neurons, but changing the regularization strength makes its final decision regions smoother with a higher regularization.

> Should not be using smaller networks because you are afraid of overfitting. Should use as big of a neural network as your computational budget allows, and use other regularization techniques to control overfitting.

## Summary  <a name='summary'></a>
- Layered architecture enables very efficient evaluation of Neural Networks based on matrix multiplications interwoven with the application of the activation function.
- Neural Networks are universal function approximators, but they are used because they make certain "right" assumptions about the functional forms of functions that come up in practice.
- Larger networks will always work better than smaller networks, but their higher model capacity must be appropriately addressed with stronger regularization (such as higher weight decay), or they might overfit.

## Reference
- [Setting up neural network architecture](http://cs231n.github.io/neural-networks-1/)